In [1]:
from tqdm import tqdm
from cobra import Reaction
import escher
import escher.urls
import cobra
import cobra.test
import json
import os
import string
from IPython.display import HTML
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis.parsimonious import optimize_minimal_flux
from cobra import ArrayBasedModel
import numpy as np
model_ref = cobra.io.load_matlab_model('C:\\Users\\Omar\\Desktop\\virtual\\venv\\RECON1', 'RECON1')

# 1) Get list of lists of unique genes and corresponding reactions

In [2]:
gprs = []
rxns = []
# get reacions in model and the associate gprs
for reaction in model_ref.reactions:
    rxns.append(reaction)
    gpr = reaction.gene_reaction_rule
    gprs.append(gpr)

In [3]:
with open('mann_biserial_cutoff_down_single.csv','r') as bis:
    clus_list = []
    ent_list = []
    genes_names_list = []
    biserial_list=[]
    log2fold_list = []
    for line in bis:
        line_ele = line.strip().split(",")
        clus_list.append(line_ele[0])
        ent_list.append(line_ele[1])
        genes_names_list.append(line_ele[2])
        biserial_list.append(line_ele[3])
        log2fold_list.append(line_ele[4])

In [4]:
unique_ent = []
for i in ent_list:
    if i not in unique_ent:
        unique_ent.append(i)

In [5]:
#rxns_dict={}
count = 0
overall_rxns_incl_down =[]
for i,x in enumerate(unique_ent):
    rxns_incl = []
    gpr_incl = []
    for reaction in model_ref.reactions:
        gpr = reaction.gene_reaction_rule
        gpr = gpr.replace(')','')
        gpr = gpr.replace('(','')
        gpr = gpr.strip().split()
        
        for item in gpr:
            if x + '_AT1' in item or x+'_AT2' in item or x + '_AT3' in item or x+'_AT4' in item or x+'_AT5' in item:
                if len(item) == len(x + '_AT1'):
                    rxns_incl.append(str(reaction))
                    gpr_incl.append(gpr)
                    #print ent,reaction, gpr
                    count +=1
    overall_rxns_incl_down.append(rxns_incl)
    #rxns_dict[ent] = rxns_incl

In [6]:
unique_rxns = []
for i in range(len(overall_rxns_incl_down)):
    cur_set = overall_rxns_incl_down[i]
    cur_uniques=[]
    for j in range(len(cur_set)):
        if cur_set[j] not in cur_uniques:
            cur_uniques.append(cur_set[j])
    unique_rxns.append(cur_uniques)

In [7]:
len(unique_rxns)

134

In [8]:
len(unique_ent)

134

# 2) Set up biomass reaction as well as RECON1 -- note: EX__arg is set at a lb of -0.22

## this is done such that a flux solution can be found in the healthy case

* create demands for: ethanalamine (lb set at -1), 
* create a transporter for Tyr_ggnt (lb set at -1000)
* change all DM_ and EX_ lower bounds to 0
* set the lower bounds to -1 of the following exceptions: EX_peplys_e, EX_Tyr_ggn_e, EX_arg__L_e, EX_asn__L_e, EX_asp__L_e, EX_chol_e, EX_cl_e, EX_glc__D_e, EX_gln__L_e, EX_gly_e, EX_h_e, EX_h2o_e, EX_ile__L_e, EX_k_e, EX_leu__L_e, EX_lys__L_e, EX_na1_e, EX_nh4_e, EX_o2_e, EX_phe__L_e, EX_pi_e, EX_pro__L_e, EX_ser__L_e, EX_thr__L_e, EX_trp__L_e, EX_tyr__L_e, EX_val__L_e
* set EX_yvite_e and EX_10fthf5glu_e to 0 as well
* add the biomass reaction as per the following
* add the biomass reaction to the model and set as model objective
* add a DM_atp_c demand DM_atp_c.add_metabolites({atp_c: -1, h2o_c: -1, adp_c: 1, pi_c: 1, h_c: 1}) with the lower bound set as 0.9*7.9939
* free the following reactions model.reactions.get_by_id('EX_h_e').lower_bound = -1000
model.reactions.get_by_id('EX_o2_e').lower_bound = -1000
model.reactions.get_by_id('EX_fe2_e').lower_bound = -1000
model.reactions.get_by_id('EX_nh4_e').lower_bound = -1000
model.reactions.get_by_id('EX_glc__D_e').lower_bound = -0.4882
model.reactions.get_by_id('EX_pi_e').lower_bound = -1000

* constrain the upper and lower bound of the biomass reaction to 0.0177, obtained from calculate fluxes

In [9]:
#creat ethanalamine demand
#metabolite is etha_c

DM_etha_c = Reaction('DM_etha_c')
DM_etha_c.name = DM_etha_c

etha_c = model_ref.metabolites.get_by_id('etha_c')

DM_etha_c.add_metabolites({etha_c: -1})
model_ref.add_reaction(DM_etha_c)

#create a transporter for this

Tyr_ggnt = Reaction('Tyr_ggnt')
Tyr_ggnt.name = Tyr_ggnt

Tyr_ggn_e = model_ref.metabolites.get_by_id('Tyr_ggn_e')
Tyr_ggn_c = model_ref.metabolites.get_by_id('Tyr_ggn_c')

Tyr_ggnt.add_metabolites({Tyr_ggn_e: -1, Tyr_ggn_c: 1})
model_ref.add_reaction(Tyr_ggnt)

DM_etha_c.lower_bound = -1
Tyr_ggnt.lower_bound = -1000

except_EX_names = '''DM_etha_c, EX_peplys_e, EX_Tyr_ggn_e, EX_arg__L_e, EX_asn__L_e, EX_asp__L_e, EX_chol_e, EX_cl_e, EX_glc__D_e, EX_gln__L_e, EX_gly_e, EX_h_e, EX_h2o_e, EX_ile__L_e, EX_k_e, EX_leu__L_e, EX_lys__L_e, EX_na1_e, EX_nh4_e, EX_o2_e, EX_phe__L_e, EX_pi_e, EX_pro__L_e, EX_ser__L_e, EX_thr__L_e, EX_trp__L_e, EX_tyr__L_e, EX_val__L_e'''
except_EX_names_split = except_EX_names.split(', ')
print except_EX_names_split

# Changing all lower bounds to 0 except the given reactions above

for a in model_ref.reactions[1188:1591]:
    a.lower_bound = 0

model_ref.reactions.get_by_id('EX_yvite_e').lower_bound = 0
model_ref.reactions.get_by_id('EX_10fthf5glu_e').lower_bound = 0
    
for b in model_ref.reactions[1188:1591]:
    for c in range(len(except_EX_names_split)):  
        if b.id  == except_EX_names_split[c]:
            b.lower_bound = -1
            
biomass_NCI60 = Reaction('biomass_NCI60')
biomass_NCI60.name = biomass_NCI60

ala_L_c = model_ref.metabolites.get_by_id('ala__L_c')
arg_L_c = model_ref.metabolites.get_by_id('arg__L_c')
asn_L_c = model_ref.metabolites.get_by_id('asn__L_c')
asp_L_c = model_ref.metabolites.get_by_id('asp__L_c')
atp_c = model_ref.metabolites.get_by_id('atp_c')
clpn_hs_c = model_ref.metabolites.get_by_id('clpn_hs_c')
ctp_c = model_ref.metabolites.get_by_id('ctp_c')
dag_hs_c = model_ref.metabolites.get_by_id('dag_hs_c')
datp_c = model_ref.metabolites.get_by_id('datp_c')
dctp_c = model_ref.metabolites.get_by_id('dctp_c')
dgtp_c = model_ref.metabolites.get_by_id('dgtp_c')
dttp_c = model_ref.metabolites.get_by_id('dttp_c')
gln_L_c = model_ref.metabolites.get_by_id('gln__L_c')
glu_L_c = model_ref.metabolites.get_by_id('glu__L_c')
gly_c = model_ref.metabolites.get_by_id('gly_c')
glygn2_c = model_ref.metabolites.get_by_id('glygn2_c')
gtp_c = model_ref.metabolites.get_by_id('gtp_c')
h2o_c = model_ref.metabolites.get_by_id('h2o_c')
hdca_c = model_ref.metabolites.get_by_id('hdca_c')
hdcea_c = model_ref.metabolites.get_by_id('hdcea_c')
ile_L_c = model_ref.metabolites.get_by_id('ile__L_c')
leu_L_c = model_ref.metabolites.get_by_id('leu__L_c')
lpchol_hs_c = model_ref.metabolites.get_by_id('lpchol_hs_c')
lys_L_c = model_ref.metabolites.get_by_id('lys__L_c')
mag_hs_c = model_ref.metabolites.get_by_id('mag_hs_c')
ocdca_c = model_ref.metabolites.get_by_id('ocdca_c')
ocdcea_c = model_ref.metabolites.get_by_id('ocdcea_c')
pa_hs_c = model_ref.metabolites.get_by_id('pa_hs_c')
pail_hs_c = model_ref.metabolites.get_by_id('pail_hs_c')
pchol_hs_c = model_ref.metabolites.get_by_id('pchol_hs_c')
pe_hs_c = model_ref.metabolites.get_by_id('pe_hs_c')
phe_L_c = model_ref.metabolites.get_by_id('phe__L_c')
ps_hs_c = model_ref.metabolites.get_by_id('ps_hs_c')
ser_L_c = model_ref.metabolites.get_by_id('ser__L_c')
sphmyln_hs_c = model_ref.metabolites.get_by_id('sphmyln_hs_c')
thr_L_c = model_ref.metabolites.get_by_id('thr__L_c')
trp_L_c = model_ref.metabolites.get_by_id('trp__L_c')
tyr_L_c = model_ref.metabolites.get_by_id('tyr__L_c')
utp_c = model_ref.metabolites.get_by_id('utp_c')
val_L_c = model_ref.metabolites.get_by_id('val__L_c')
pro_L_m = model_ref.metabolites.get_by_id('pro__L_m')
chsterol_r = model_ref.metabolites.get_by_id('chsterol_r')
xolest_hs_r = model_ref.metabolites.get_by_id('xolest_hs_r')
adp_c = model_ref.metabolites.get_by_id('adp_c')
h_c = model_ref.metabolites.get_by_id('h_c')
pi_c = model_ref.metabolites.get_by_id('pi_c')

#remove ocdca and ocdcea
# added xolest value to cholesterol in biomass and remove xolest

biomass_NCI60.add_metabolites({ala_L_c: -0.587929, arg_L_c: -0.380280, asn_L_c: -0.323313, asp_L_c: -0.261396, 
                               atp_c: -35.033540, clpn_hs_c: -0.000624, ctp_c: -0.033435, dag_hs_c: -0.001032, 
                               datp_c: -0.014557, dctp_c: -0.009770, dgtp_c: -0.009748, dttp_c: -0.014546,
                               gln_L_c: -0.319051, glu_L_c: -0.387401, gly_c: -0.504294, glygn2_c: -0.034479,
                               gtp_c: -0.055967, h2o_c: -35.000000, hdca_c: -0.008293, hdcea_c: -0.003315, 
                               ile_L_c: -0.319813, leu_L_c: -0.548692, lpchol_hs_c: -0.002470, lys_L_c: -0.552717, 
                               mag_hs_c: -0.001456, pa_hs_c: -0.010645,
                               pail_hs_c: -0.005016, pchol_hs_c: -0.022878, pe_hs_c: -0.018211, phe_L_c: -0.170743,
                               ps_hs_c: -0.006808, ser_L_c: -0.385852, sphmyln_hs_c: -0.010215, thr_L_c: -0.378004, 
                               trp_L_c: -0.039847, tyr_L_c: -0.150141, utp_c: -0.063323, val_L_c: -0.385554,
                               pro_L_m: -0.237850, chsterol_r: -0.054102,  adp_c: 35.000000, 
                               h_c: 35.000000, pi_c: 35.000000})

model_ref.add_reaction(biomass_NCI60)
my_objective = model_ref.reactions.get_by_id('biomass_NCI60')
model_ref.change_objective(my_objective)

# Adding DM_atp_c reaction

DM_atp_c = Reaction('DM_atp_c')
DM_atp_c.name = DM_atp_c

atp_c = model_ref.metabolites.get_by_id('atp_c')
adp_c = model_ref.metabolites.get_by_id('adp_c')
h2o_c = model_ref.metabolites.get_by_id('h2o_c')
h_c = model_ref.metabolites.get_by_id('h_c')
pi_c = model_ref.metabolites.get_by_id('pi_c')


DM_atp_c.add_metabolites({atp_c: -1, h2o_c: -1, adp_c: 1, pi_c: 1, h_c: 1})
model_ref.add_reaction(DM_atp_c)

# Setting lower bounds for DM_atp_c as per Dan

model_ref.reactions.get_by_id('DM_atp_c').lower_bound = 0.9*7.9939

model_ref.reactions.get_by_id('EX_h_e').lower_bound = -1000
model_ref.reactions.get_by_id('EX_o2_e').lower_bound = -1000
model_ref.reactions.get_by_id('EX_fe2_e').lower_bound = -1000
model_ref.reactions.get_by_id('EX_nh4_e').lower_bound = -1000

model_ref.reactions.get_by_id('EX_glc__D_e').lower_bound = -0.4882

# constrained as per calculate fluxes 
model_ref.reactions.get_by_id('biomass_NCI60').lower_bound = 0.0177
model_ref.reactions.get_by_id('biomass_NCI60').upper_bound = 0.0177

model_ref.reactions.get_by_id('EX_pi_e').lower_bound = -1000

# 6/16/16 Changing amino acid uptake constraints
# arg__L__e initially was -0.0022
model_ref.reactions.get_by_id('EX_arg__L_e').lower_bound = -0.0022

model_ref.reactions.get_by_id('EX_asn__L_e').lower_bound = -0.0058
model_ref.reactions.get_by_id('EX_asp__L_e').lower_bound = -0.0081
model_ref.reactions.get_by_id('EX_chol_e').lower_bound = -0.00062386
model_ref.reactions.get_by_id('EX_gln__L_e').lower_bound = -0.1202
model_ref.reactions.get_by_id('EX_gly_e').lower_bound = -0.0019
model_ref.reactions.get_by_id('EX_ile__L_e').lower_bound = -0.0081
model_ref.reactions.get_by_id('EX_leu__L_e').lower_bound = -0.0110
model_ref.reactions.get_by_id('EX_lys__L_e').lower_bound = -0.0111
model_ref.reactions.get_by_id('EX_phe__L_e').lower_bound = -0.0040

model_ref.reactions.get_by_id('EX_pro__L_e').lower_bound = -0.0045

model_ref.reactions.get_by_id('EX_ser__L_e').lower_bound = -0.0167
model_ref.reactions.get_by_id('EX_trp__L_e').lower_bound = -0.0013
model_ref.reactions.get_by_id('EX_tyr__L_e').lower_bound = -0.0068
model_ref.reactions.get_by_id('EX_val__L_e').lower_bound = -0.0079

#6/24/16 lowerbound changes

model_ref.reactions.get_by_id('FTHFLi').lower_bound = 0
model_ref.reactions.get_by_id('FTHFLmi').lower_bound = 0

model_ref.reactions.get_by_id('FUM').lower_bound = 0
model_ref.reactions.get_by_id('FUM').upper_bound = 0

model_ref.reactions.get_by_id('BILGLCURte').lower_bound = 0
model_ref.reactions.get_by_id('BILDGLCURte').lower_bound = 0

h_i = Metabolite('h_i', formula='h', name='h_i', compartment='i')
model_ref.add_metabolites(h_i)


# delete reaction
model_ref.remove_reactions('ATPS4m')

# re-add the reaction while replacing h_c with h_i
ATPS4m = Reaction('ATPS4m')
ATPS4m.name = ATPS4m

adp_m = model_ref.metabolites.get_by_id('adp_m')
h_i = model_ref.metabolites.get_by_id('h_i')
pi_m = model_ref.metabolites.get_by_id('pi_m')
h2o_m = model_ref.metabolites.get_by_id('h2o_m')
h_m = model_ref.metabolites.get_by_id('h_m')
atp_m = model_ref.metabolites.get_by_id('atp_m')

ATPS4m.add_metabolites({adp_m: -1, h_i: -4, pi_m: -1, h2o_m: 1, h_m: 3, atp_m: 1})
model_ref.add_reaction(ATPS4m)

model_ref.remove_reactions('CYOOm3')

CYOOm3 = Reaction('CYOOm3')
CYOOm3.name = CYOOm3

h_m = model_ref.metabolites.get_by_id('h_m')
o2_m = model_ref.metabolites.get_by_id('o2_m')
focytC_m = model_ref.metabolites.get_by_id('focytC_m')
h_i = model_ref.metabolites.get_by_id('h_i')
h2o_m = model_ref.metabolites.get_by_id('h2o_m')
ficytC_m = model_ref.metabolites.get_by_id('ficytC_m')


CYOOm3.add_metabolites({h_m: -8, o2_m: -1, focytC_m: -4, h_i: 4, h2o_m: 2, ficytC_m: 4})
model_ref.add_reaction(CYOOm3)

# delete reaction
model_ref.remove_reactions('NADH2_u10m')

# re-add the reaction while replacing h_c with h_i
NADH2_u10m = Reaction('NADH2_u10m')
NADH2_u10m.name = NADH2_u10m

q10_m = model_ref.metabolites.get_by_id('q10_m')
h_m = model_ref.metabolites.get_by_id('h_m')
nadh_m = model_ref.metabolites.get_by_id('nadh_m')
nad_m = model_ref.metabolites.get_by_id('nad_m')
h_i = model_ref.metabolites.get_by_id('h_i')
q10h2_m = model_ref.metabolites.get_by_id('q10h2_m')



NADH2_u10m.add_metabolites({q10_m: -1, h_m: -5, nadh_m: -1, nad_m: 1, h_i: 4, q10h2_m: 1})
model_ref.add_reaction(NADH2_u10m)

# delete reaction
model_ref.remove_reactions('CYOR_u10m')

# re-add the reaction while replacing h_c with h_i
CYOR_u10m = Reaction('CYOR_u10m')
CYOR_u10m.name = NADH2_u10m

ficytC_m = model_ref.metabolites.get_by_id('ficytC_m')
h_m = model_ref.metabolites.get_by_id('h_m')
q10h2_m = model_ref.metabolites.get_by_id('q10h2_m')
q10_m = model_ref.metabolites.get_by_id('q10_m')
focytC_m = model_ref.metabolites.get_by_id('focytC_m')
h_i = model_ref.metabolites.get_by_id('h_i')


CYOR_u10m.add_metabolites({ficytC_m: -2, h_m: -2, q10h2_m: -1, q10_m: 1, focytC_m: 2, h_i: 4})
model_ref.add_reaction(CYOR_u10m)

['DM_etha_c', 'EX_peplys_e', 'EX_Tyr_ggn_e', 'EX_arg__L_e', 'EX_asn__L_e', 'EX_asp__L_e', 'EX_chol_e', 'EX_cl_e', 'EX_glc__D_e', 'EX_gln__L_e', 'EX_gly_e', 'EX_h_e', 'EX_h2o_e', 'EX_ile__L_e', 'EX_k_e', 'EX_leu__L_e', 'EX_lys__L_e', 'EX_na1_e', 'EX_nh4_e', 'EX_o2_e', 'EX_phe__L_e', 'EX_pi_e', 'EX_pro__L_e', 'EX_ser__L_e', 'EX_thr__L_e', 'EX_trp__L_e', 'EX_tyr__L_e', 'EX_val__L_e']


cobra\core\Model.py:289 UserWarning: need to pass in a list


# 3) Run pFBA with biomass set as the objective

## Calculating the normal, healthy demands

In [10]:
p_solution_ref = cobra.flux_analysis.parsimonious.optimize_minimal_flux(model_ref)

In [32]:
p_solution_ref.x_dict['SUCD1m']

0.4979664887656241

In [33]:
p_solution_ref.f

0.0177

## Check for conserved moieties

In [35]:
model_mo = model_ref.copy()

# creates an empty dict to store DM_rxns as keys and the p_solution.f values
KO_maxmet_dict = {}


newmet_array = [0] * len(model_mo.metabolites)
met_count = 0
for i in range(len(model_mo.metabolites)):
    newmet_array[met_count] = string.replace(model_mo.metabolites[i].id, " ", "_")
    met_count = met_count + 1

conserved_moieties = []

for i in tqdm(range(len(model_mo.metabolites))):
    #for i in range(80):

    #  switch toggle
    switch = True

    # this part creates a demand reaction for each metabolite
    temp_DM = 'DM_' + str(newmet_array[i])

    # this nested for loop tries to catch if a demand reaction already exists in the model
    for ii in model_mo.reactions:
        if temp_DM == str(ii):
            # this part sets the newly created demand as the objective
            my_objective = model_mo.reactions.get_by_id(str(temp_DM))
            model_mo.change_objective(my_objective)

            # this part runs FBA on the new demand
            p_solution = model_mo.optimize(solver='gurobi')
            
            KO_maxmet_dict[str(temp_DM)] = p_solution.f
            
            # check if it can be produced, if it can't add it to the list so when algorithm is run we dont include it

            if -10**-9< p_solution < 10**-9:
                conserved_moieties.append(str(temp_DM))

            # a toggle to see if the reaction is to be deleted or not
            switch = False

    if switch is True:
        temp_DM = Reaction(str(temp_DM))
        newmet_array[i] = model_mo.metabolites.get_by_id(model_mo.metabolites[i].id)
        temp_DM.add_metabolites({newmet_array[i]: -1})
        model_mo.add_reaction(temp_DM)

        # this part sets the newly created demand as the objective
        my_objective = model_mo.reactions.get_by_id(str(temp_DM))
        model_mo.change_objective(my_objective)

        # this part runs FBA on the new demand
        p_solution = model_mo.optimize(solver='gurobi')
        
        KO_maxmet_dict[str(temp_DM)] = p_solution.f

        # check if it can be produced, if it can't add it to the list so when algorithm is run we dont include it

        if -10**-9< p_solution < 10**-9:
            conserved_moieties.append(str(temp_DM))

        # this deletes the demand at the end of each iteration
        model_mo.reactions.get_by_id(str(temp_DM)).delete()

100%|██████████████████████████████████████████████████████████████████████████████| 2767/2767 [07:05<00:00,  5.76it/s]


In [36]:
conserved_moieties

[]

In [42]:
KO_maxmet_dict['DM_nadh_m']

-5.355853954780443e-11

In [ ]:
10**-9

In [ ]:
# create a copy of reference model so don't have to worry about things accidently adding up
model = model_ref.copy()
print model.optimize()
#create a demand of the products of the corresponding reactions mapped by the gene
# keep track of the reactions we added to this model so we can delete them for the next round
# alternatively could do something like model_temp = model.copy() or model = model_ref.copy()
reactions_added = []
unique_rxns = ['FUMm']
# for each reaction mapped by the gene
for reaction in unique_rxns[i]:
    model.reactions.get_by_id(reaction).lower_bound = p_solution_ref.x_dict[reaction]
    #out.write('%s,%s,%s\n'%(unique_ent[i],reaction,p_solution.x_dict[reaction]))
    #print reaction
    # iterate through products of the reaction
    if p_solution_ref.x_dict[reaction] > 10**-9:
        for x in model.reactions.get_by_id(str(reaction)).reactants:
            # create string for demand for further use
            temp_DM = 'DM_' + str(x)
            
            for item in conserved_moieties:
                if temp_DM == item:
                    pass
                else:

                    # create string for metabolite for futher use
                    temp_met = model.metabolites.get_by_id(str(x))
                    #print temp_met

                    # handy toggle
                    switch = True

                    # check if demand reaction already in model
                    for ii in model.reactions:
                        # if already in model add the new lower bound to previous lower bound
                        if temp_DM == str(ii):
                            model.reactions.get_by_id(str(temp_DM)).lower_bound = model.reactions.get_by_id(str(temp_DM)).lower_bound + (p_solution_ref.x_dict[str(reaction)])
                            switch = False
                    # if demand not already in the model create the demand and set lower bound to pFBA solution of the reaction
                    if switch == True:
                        temp_DM = Reaction(str(temp_DM))
                        temp_DM.add_metabolites({temp_met: -1})
                        model.add_reaction(temp_DM)
                        model.reactions.get_by_id(str(temp_DM)).lower_bound = 0.0001
                        model.reactions.get_by_id(str(temp_DM)).upper_bound = 0.0001
                        reactions_added.append(str(temp_DM))
    elif p_solution_ref.x_dict[reaction] < -10**-9:
        for x in model.reactions.get_by_id(str(reaction)).products:
            # create string for demand for further use
            temp_DM = 'DM_' + str(x)
            
            for item in conserved_moieties:
                if temp_DM == item:
                    pass
                else:
                    # create string for metabolite for futher use
                    temp_met = model.metabolites.get_by_id(str(x))
                    #print temp_met

                    # handy toggle
                    switch = True

                    # check if demand reaction already in model
                    for ii in model.reactions:
                        # if already in model add the new lower bound to previous lower bound
                        if temp_DM == str(ii):
                            model.reactions.get_by_id(str(temp_DM)).lower_bound = model.reactions.get_by_id(str(temp_DM)).lower_bound - p_solution_ref.x_dict[str(reaction)]
                            switch = False
                    # if demand not already in the model create the demand and set lower bound to pFBA solution of the reaction
                    if switch == True:
                        temp_DM = Reaction(str(temp_DM))
                        temp_DM.add_metabolites({temp_met: -1})
                        model.add_reaction(temp_DM)
                        model.reactions.get_by_id(str(temp_DM)).lower_bound = 0.0001
                        model.reactions.get_by_id(str(temp_DM)).upper_bound = 0.0001
                        reactions_added.append(str(temp_DM))

In [ ]:
#####ALGORITHM################
# creates an empty dict to store DM_rxns as keys and the p_solution.f values
normal_maxmet_dict = {}

model_alg model.copy()

newmet_array = [0] * len(model_alg.metabolites)
met_count = 0
for i in range(len(model_alg.metabolites)):
    newmet_array[met_count] = string.replace(model_alg.metabolites[i].id, " ", "_")
    met_count = met_count + 1

for i in range(len(model_alg.metabolites)):
    #for i in range(80):

    #  switch toggle
    switch = True

    # this part creates a demand reaction for each metabolite
    temp_DM = 'DM_' + str(newmet_array[i])

    # this nested for loop tries to catch if a demand reaction already exists in the model
    for ii in model_alg.reactions:
        if temp_DM == str(ii):
            # this part sets the newly created demand as the objective
            my_objective = model_alg.reactions.get_by_id(str(temp_DM))
            model_alg.change_objective(my_objective)

            # this part runs FBA on the new demand
            p_solution = model_alg.optimize(solver='gurobi')

            # this stores the demand flux into a dictionary where its demand rxn is the key
            normal_maxmet_dict[str(temp_DM)] = p_solution.f

            # a toggle to see if the reaction is to be deleted or not
            switch = False

    if switch is True:
        temp_DM = Reaction(str(temp_DM))
        newmet_array[i] = model_alg.metabolites.get_by_id(model_alg.metabolites[i].id)
        temp_DM.add_metabolites({newmet_array[i]: -1})
        model_alg.add_reaction(temp_DM)

        # this part sets the newly created demand as the objective
        my_objective = model_alg.reactions.get_by_id(str(temp_DM))
        model_alg.change_objective(my_objective)

        # this part runs FBA on the new demand
        p_solution = model_alg.optimize(solver='gurobi')

        # this stores the demand flux into a dictionary where its demand rxn is the key
        normal_maxmet_dict[str(temp_DM)] = p_solution.f

        # this deletes the demand at the end of each iteration
        model_alg.reactions.get_by_id(str(temp_DM)).delete()

In [14]:
# set up a cutoff 
for rxn in model_ref.reactions:
    if -10**-6 < p_solution_ref.x_dict[str(rxn)] < 10**-6 and p_solution_ref.x_dict[str(rxn)] != 0 :
        p_solution_ref.x_dict[str(rxn)] = 0
        

In [15]:
model_ref.objective

{<Reaction biomass_NCI60 at 0x11438668>: 1}

In [18]:
model = ArrayBasedModel(model_ref)
S = model.S

In [19]:
# initialize with 0 in order for this method to work
p = [0]
cgh = np.zeros(len(model.metabolites))
cgh = np.matrix(cgh)

for i in range(len(model.reactions)):
    sva = []
    for h in range(len(model.metabolites)):
        sva.append(S[h,i])
    sva = np.array(sva)    
    sva = np.matrix(sva.T)
    cgh = np.concatenate((cgh,sva))

In [30]:
import numpy as np
from numpy.linalg import svd
def nullspace(A, atol=1e-13, rtol=0):
    """Compute an approximate basis for the nullspace of A.

    The algorithm used by this function is based on the singular value
    decomposition of `A`.

    Parameters
    ----------
    A : ndarray
        A should be at most 2-D.  A 1-D array with length k will be treated
        as a 2-D with shape (1, k)
    atol : float
        The absolute tolerance for a zero singular value.  Singular values
        smaller than `atol` are considered to be zero.
    rtol : float
        The relative tolerance.  Singular values less than rtol*smax are
        considered to be zero, where smax is the largest singular value.

    If both `atol` and `rtol` are positive, the combined tolerance is the
    maximum of the two; that is::
        tol = max(atol, rtol * smax)
    Singular values smaller than `tol` are considered to be zero.

    Return value
    ------------
    ns : ndarray
        If `A` is an array with shape (m, k), then `ns` will be an array
        with shape (k, n), where n is the estimated dimension of the
        nullspace of `A`.  The columns of `ns` are a basis for the
        nullspace; each element in numpy.dot(A, ns) will be approximately
        zero.
    """

    A = np.atleast_2d(A)
    u, s, vh = svd(A)
    tol = max(atol, rtol * s[0])
    nnz = (s >= tol).sum()
    ns = vh[nnz:].conj().T
    return ns

In [32]:
out = nullspace(cgh)

In [96]:
#unique = [['ARGSS']]

In [130]:
out = open('flux_reactions2.csv','w')
#out_file = open('infeasible_reactions.txt','w')

In [38]:
unique_rxns = [['SUCD1m']]

In [41]:
model.reactions.get_by_id('SUCD1m').reaction

'fad_m + succ_m <=> fadh2_m + fum_m'

In [93]:
run ={}
final ={}

# essentially for each gene to be knocked out
for i in range(len(unique_rxns)):
    # create a copy of reference model so don't have to worry about things accidently adding up
    model = model_ref.copy()
    print model.optimize()
    #create a demand of the products of the corresponding reactions mapped by the gene
    # keep track of the reactions we added to this model so we can delete them for the next round
    # alternatively could do something like model_temp = model.copy() or model = model_ref.copy()
    reactions_added = []
    # for each reaction mapped by the gene
    for reaction in unique_rxns[i]:
        model.reactions.get_by_id(reaction).lower_bound = p_solution_ref.x_dict[reaction]
        #out.write('%s,%s,%s\n'%(unique_ent[i],reaction,p_solution.x_dict[reaction]))
        #print reaction
        # iterate through products of the reaction
        if p_solution_ref.x_dict[reaction] > 10**-9:
            for x in model.reactions.get_by_id(str(reaction)).reactants:
                # create string for demand for further use
                temp_DM = 'DM_' + str(x)
                # create string for metabolite for futher use
                temp_met = model.metabolites.get_by_id(str(x))
                #print temp_met

                # handy toggle
                switch = True

                # check if demand reaction already in model
                for ii in model.reactions:
                    # if already in model add the new lower bound to previous lower bound
                    if temp_DM == str(ii):
                        model.reactions.get_by_id(str(temp_DM)).lower_bound = model.reactions.get_by_id(str(temp_DM)).lower_bound + (p_solution_ref.x_dict[str(reaction)])
                        switch = False
                # if demand not already in the model create the demand and set lower bound to pFBA solution of the reaction
                if switch == True:
                    temp_DM = Reaction(str(temp_DM))
                    temp_DM.add_metabolites({temp_met: -1})
                    model.add_reaction(temp_DM)
                    model.reactions.get_by_id(str(temp_DM)).lower_bound = p_solution_ref.x_dict[str(reaction)]
                    reactions_added.append(str(temp_DM))
        elif p_solution_ref.x_dict[reaction] < -10**-9:
            for x in model.reactions.get_by_id(str(reaction)).products:
                # create string for demand for further use
                temp_DM = 'DM_' + str(x)
                # create string for metabolite for futher use
                temp_met = model.metabolites.get_by_id(str(x))
                #print temp_met

                # handy toggle
                switch = True

                # check if demand reaction already in model
                for ii in model.reactions:
                    # if already in model add the new lower bound to previous lower bound
                    if temp_DM == str(ii):
                        model.reactions.get_by_id(str(temp_DM)).lower_bound = model.reactions.get_by_id(str(temp_DM)).lower_bound - p_solution_ref.x_dict[str(reaction)]
                        switch = False
                # if demand not already in the model create the demand and set lower bound to pFBA solution of the reaction
                if switch == True:
                    temp_DM = Reaction(str(temp_DM))
                    temp_DM.add_metabolites({temp_met: -1})
                    model.add_reaction(temp_DM)
                    model.reactions.get_by_id(str(temp_DM)).lower_bound = -p_solution_ref.x_dict[str(reaction)]
                    reactions_added.append(str(temp_DM))
        print x,model.optimize()

<Solution 0.02 at 0x1ac1d908>
fum_m <Solution 'infeasible' at 0x1b804908>


In [104]:
model = model_ref.copy()

In [105]:
model.reactions.get_by_id('SUCD1m').reaction

'fad_m + succ_m <=> fadh2_m + fum_m'

In [106]:
model.reactions.get_by_id('SUCD1m').lower_bound = p_solution_ref.x_dict['SUCD1m']

In [107]:
model.optimize()

<Solution 0.02 at 0x1925dc50>

In [108]:
fadh2_m = model.metabolites.get_by_id('fadh2_m')
fum_m = model.metabolites.get_by_id('fum_m')

In [109]:
DM_fadh2_m = Reaction('DM_fadh2_m')
DM_fadh2_m.add_metabolites({fadh2_m:-1})
model.add_reaction(DM_fadh2_m)

In [129]:
model.reactions.get_by_id('DM_fadh2_m').lower_bound = 10**-8

In [130]:
model.change_objective('DM_fadh2_m')

In [131]:
model.optimize().f

-5.767047423944384e-11

In [37]:
reaction

'NACHEXA11ly'

In [20]:
model.optimize()

<Solution 'infeasible' at 0x10ac43c8>

In [14]:
run ={}
final ={}

# essentially for each gene to be knocked out
for i in range(len(unique_rxns)):
    # create a copy of reference model so don't have to worry about things accidently adding up
    model = model_ref.copy()
    #create a demand of the products of the corresponding reactions mapped by the gene
    # keep track of the reactions we added to this model so we can delete them for the next round
    # alternatively could do something like model_temp = model.copy() or model = model_ref.copy()
    reactions_added = []
    # for each reaction mapped by the gene
    for reaction in unique_rxns[i]:
        #out.write('%s,%s,%s\n'%(unique_ent[i],reaction,p_solution.x_dict[reaction]))
        #print reaction
        # iterate through products of the reaction
        if p_solution_ref.x_dict[reaction] > 0:
            for x in model.reactions.get_by_id(str(reaction)).products:
                # create string for demand for further use
                temp_DM = 'DM_' + str(x)
                # create string for metabolite for futher use
                temp_met = model.metabolites.get_by_id(str(x))
                #print temp_met

                # handy toggle
                switch = True

                # check if demand reaction already in model
                for ii in model.reactions:
                    # if already in model add the new lower bound to previous lower bound
                    if temp_DM == str(ii):
                        model.reactions.get_by_id(str(temp_DM)).lower_bound = model.reactions.get_by_id(str(temp_DM)).lower_bound + p_solution_ref.x_dict[str(reaction)]
                        switch = False
                # if demand not already in the model create the demand and set lower bound to pFBA solution of the reaction
                if switch == True:
                    temp_DM = Reaction(str(temp_DM))
                    temp_DM.add_metabolites({temp_met: -1})
                    model.add_reaction(temp_DM)
                    model.reactions.get_by_id(str(temp_DM)).lower_bound = p_solution_ref.x_dict[str(reaction)]
                    reactions_added.append(str(temp_DM))
        elif p_solution_ref.x_dict[reaction] < 0:
            for x in model.reactions.get_by_id(str(reaction)).reactants:
                # create string for demand for further use
                temp_DM = 'DM_' + str(x)
                # create string for metabolite for futher use
                temp_met = model.metabolites.get_by_id(str(x))
                #print temp_met

                # handy toggle
                switch = True

                # check if demand reaction already in model
                for ii in model.reactions:
                    # if already in model add the new lower bound to previous lower bound
                    if temp_DM == str(ii):
                        model.reactions.get_by_id(str(temp_DM)).lower_bound = model.reactions.get_by_id(str(temp_DM)).lower_bound - p_solution_ref.x_dict[str(reaction)]
                        switch = False
                # if demand not already in the model create the demand and set lower bound to pFBA solution of the reaction
                if switch == True:
                    temp_DM = Reaction(str(temp_DM))
                    temp_DM.add_metabolites({temp_met: -1})
                    model.add_reaction(temp_DM)
                    model.reactions.get_by_id(str(temp_DM)).lower_bound = -p_solution_ref.x_dict[str(reaction)]
                    reactions_added.append(str(temp_DM))
        #else:
            #out.write('%s,%s,%s\n'%(unique_ent[i],reaction,p_solution.x_dict[reaction]))
     
    ###### CREATE KO MODEL ######
    model_KO = model.copy()
    for reaction in unique_rxns[i]:
        model_KO.reactions.get_by_id(reaction).lower_bound = 0
        model_KO.reactions.get_by_id(reaction).upper_bound = 0

    for reaction in reactions_added:
        model_KO.reactions.get_by_id(reaction).lower_bound = 0
        model_KO.reactions.get_by_id(reaction).upper_bound = 0
        
    ###### SHADOW PRICE ANALYSIS ######
    model_sp = model_KO.copy()
    model_sp.reactions.get_by_id('biomass_NCI60').lower_bound = 0
    model_sp.optimize(solver='gurobi').f
    # get shadow price values for each metabolite
    met_sp = []
    try:
        for met in model_sp.metabolites:
            met_sp.append(met.y)
    except Exception:
        model_sp.change_objective('DM_atp_c')
        model_sp.optimize(solver='gurobi').f
        for met in model_sp.metabolites:
            met_sp.append(met.y)
        print unique_rxns[i]
    # setup a zero cutoff
    for ind,item in enumerate(met_sp):
        if -10**-6<item<10**-6:
            met_sp[ind] = 0
    # get list of biomass metabolites
    biomass_mets = []
    for met in model_sp.reactions.get_by_id('biomass_NCI60').metabolites:
        biomass_mets.append(str(met))
        
    # get a list of the non-zero shadow prices and cross-reference biomass metabolites to find limiting nutrient(s) if any
    possible_limiting_nutrients = []
    limiting_nutrients = []
    for ind,item in enumerate(met_sp):
        if item !=0:
            possible_limiting_nutrients.append(str(model_sp.metabolites[ind]))
            possible = str(model_sp.metabolites[ind])
            if possible in biomass_mets:
                limiting_nutrients.append(possible)
    # get a list of new objective reactions which should be the corresponding exchange reaction(s) if any
    if limiting_nutrients:
        new_objective_reactions = []
        for item in limiting_nutrients:
            item = item.replace(item[-1],'e')
            item = 'EX_' + item
            new_objective_reactions.append(item)
        run[unique_ent[i]] = new_objective_reactions   
        #### DETERMINING NEW UPTAKE ####
        for n in range(len(new_objective_reactions)):
            model_new_up = model_KO.copy()
            model_new_up.objective = model.reactions.get_by_id(new_objective_reactions[n])
            model_new_up.reactions.get_by_id(new_objective_reactions[n]).lower_bound = -1000
            new_up = model_new_up.optimize(solver='gurobi').f
            model.reactions.get_by_id(new_objective_reactions[n]).lower_bound = new_up
            
            
    newmet_array = [0] * len(model.metabolites)
    met_count = 0
    for i in range(len(model.metabolites)):
        newmet_array[met_count] = string.replace(model.metabolites[i].id, " ", "_")
        met_count = met_count + 1
        
    #####ALGORITHM################
    # creates an empty dict to store DM_rxns as keys and the p_solution.f values
    normal_maxmet_dict = {}

    for i in range(len(model.metabolites)):
        #for i in range(80):

        #  switch toggle
        switch = True

        # this part creates a demand reaction for each metabolite
        temp_DM = 'DM_' + str(newmet_array[i])

        # this nested for loop tries to catch if a demand reaction already exists in the model
        for ii in model.reactions:
            if temp_DM == str(ii):
                # this part sets the newly created demand as the objective
                my_objective = model.reactions.get_by_id(str(temp_DM))
                model.change_objective(my_objective)

                # this part runs FBA on the new demand
                p_solution = model.optimize(solver='gurobi')

                # this stores the demand flux into a dictionary where its demand rxn is the key
                normal_maxmet_dict[str(temp_DM)] = p_solution.f

                # a toggle to see if the reaction is to be deleted or not
                switch = False

        if switch is True:
            temp_DM = Reaction(str(temp_DM))
            newmet_array[i] = model.metabolites.get_by_id(model.metabolites[i].id)
            temp_DM.add_metabolites({newmet_array[i]: -1})
            model.add_reaction(temp_DM)

            # this part sets the newly created demand as the objective
            my_objective = model.reactions.get_by_id(str(temp_DM))
            model.change_objective(my_objective)

            # this part runs FBA on the new demand
            p_solution = model.optimize(solver='gurobi')

            # this stores the demand flux into a dictionary where its demand rxn is the key
            normal_maxmet_dict[str(temp_DM)] = p_solution.f

            # this deletes the demand at the end of each iteration
            model.reactions.get_by_id(str(temp_DM)).delete()

    
    
    ####KO######
    
    p_solution = cobra.flux_analysis.parsimonious.optimize_minimal_flux(model_KO)
    
    
    # working copy!
    # This creates an array of modified met names since I'm getting errors accessing model.metabolites[i].id in some of the code
    newmet_array = [0] * len(model_KO.metabolites)
    met_count = 0
    for i in range(len(model_KO.metabolites)):
        newmet_array[met_count] = string.replace(model_KO.metabolites[i].id, " ", "_")
        met_count = met_count + 1
        
        
    # creates an empty dict to store DM_rxns as keys and the p_solution.f values
    KO_maxmet_dict = {}

    for i in range(len(model_KO.metabolites)):
    #for i in range(80):

        # switch toggle
        switch = True

        # this part creates a demand reaction for each metabolite
        temp_DM = 'DM_' + str(newmet_array[i])

        # this nested for loop tries to catch if a demand reaction already exists in the model
        for ii in model_KO.reactions:
            if temp_DM == str(ii):
                # this part sets the newly created demand as the objective
                my_objective = model_KO.reactions.get_by_id(str(temp_DM))
                model_KO.change_objective(my_objective)

                # this part runs FBA on the new demand
                p_solution = model_KO.optimize(solver='gurobi')

                # this stores the demand flux into a dictionary where its demand rxn is the key
                KO_maxmet_dict[str(temp_DM)] = p_solution.f

                # a toggle to see if the reaction is to be deleted or not
                switch = False

        if switch is True:
            temp_DM = Reaction(str(temp_DM))
            newmet_array[i] = model_KO.metabolites.get_by_id(model_KO.metabolites[i].id)
            temp_DM.add_metabolites({newmet_array[i]: -1})
            model_KO.add_reaction(temp_DM)

            # this part sets the newly created demand as the objective
            my_objective = model_KO.reactions.get_by_id(str(temp_DM))
            model_KO.change_objective(my_objective)

            # this part runs FBA on the new demand
            p_solution = model_KO.optimize(solver='gurobi')

            # this stores the demand flux into a dictionary where its demand rxn is the key
            KO_maxmet_dict[str(temp_DM)] = p_solution.f

            # this deletes the demand at the end of each iteration
            model_KO.reactions.get_by_id(str(temp_DM)).delete()
    
    #try:
        #p_solution = cobra.flux_analysis.parsimonious.optimize_minimal_flux(model)
    #except ValueError:
        #out_file.write('%s\t%s\t%s\n'%(unique_ent[i],unique_rxns[i],reactions_added))
        
#out.close()
#out_file.close()

In [26]:
run ={}
final ={}

# essentially for each gene to be knocked out
for i in range(len(unique_rxns)):
    # create a copy of reference model so don't have to worry about things accidently adding up
    model = model_ref.copy()
    #create a demand of the products of the corresponding reactions mapped by the gene
    # keep track of the reactions we added to this model so we can delete them for the next round
    # alternatively could do something like model_temp = model.copy() or model = model_ref.copy()
    reactions_added = []
    # for each reaction mapped by the gene
    for reaction in unique_rxns[i]:
        #out.write('%s,%s,%s\n'%(unique_ent[i],reaction,p_solution.x_dict[reaction]))
        #print reaction
        # iterate through products of the reaction
        if p_solution_ref.x_dict[reaction] > 0:
            for x in model.reactions.get_by_id(str(reaction)).products:
                # create string for demand for further use
                temp_DM = 'DM_' + str(x)
                # create string for metabolite for futher use
                temp_met = model.metabolites.get_by_id(str(x))
                #print temp_met

                # handy toggle
                switch = True

                # check if demand reaction already in model
                for ii in model.reactions:
                    # if already in model add the new lower bound to previous lower bound
                    if temp_DM == str(ii):
                        model.reactions.get_by_id(str(temp_DM)).lower_bound = model.reactions.get_by_id(str(temp_DM)).lower_bound + p_solution_ref.x_dict[str(reaction)]
                        switch = False
                # if demand not already in the model create the demand and set lower bound to pFBA solution of the reaction
                if switch == True:
                    temp_DM = Reaction(str(temp_DM))
                    temp_DM.add_metabolites({temp_met: -1})
                    model.add_reaction(temp_DM)
                    model.reactions.get_by_id(str(temp_DM)).lower_bound = p_solution_ref.x_dict[str(reaction)]
                    reactions_added.append(str(temp_DM))
        elif p_solution_ref.x_dict[reaction] < 0:
            for x in model.reactions.get_by_id(str(reaction)).reactants:
                # create string for demand for further use
                temp_DM = 'DM_' + str(x)
                # create string for metabolite for futher use
                temp_met = model.metabolites.get_by_id(str(x))
                #print temp_met

                # handy toggle
                switch = True

                # check if demand reaction already in model
                for ii in model.reactions:
                    # if already in model add the new lower bound to previous lower bound
                    if temp_DM == str(ii):
                        model.reactions.get_by_id(str(temp_DM)).lower_bound = model.reactions.get_by_id(str(temp_DM)).lower_bound - p_solution_ref.x_dict[str(reaction)]
                        switch = False
                # if demand not already in the model create the demand and set lower bound to pFBA solution of the reaction
                if switch == True:
                    temp_DM = Reaction(str(temp_DM))
                    temp_DM.add_metabolites({temp_met: -1})
                    model.add_reaction(temp_DM)
                    model.reactions.get_by_id(str(temp_DM)).lower_bound = -p_solution_ref.x_dict[str(reaction)]
                    reactions_added.append(str(temp_DM))
        #else:
            #out.write('%s,%s,%s\n'%(unique_ent[i],reaction,p_solution.x_dict[reaction]))
     
    ###### CREATE KO MODEL ######
    model_KO = model.copy()
    for reaction in unique_rxns[i]:
        model_KO.reactions.get_by_id(reaction).lower_bound = 0
        model_KO.reactions.get_by_id(reaction).upper_bound = 0

    for reaction in reactions_added:
        model_KO.reactions.get_by_id(reaction).lower_bound = 0
        model_KO.reactions.get_by_id(reaction).upper_bound = 0
        
    ###### SHADOW PRICE ANALYSIS ######
    model_sp = model_KO.copy()
    model_sp.reactions.get_by_id('biomass_NCI60').lower_bound = 0
    model_sp.optimize(solver='gurobi').f
    # get shadow price values for each metabolite
    met_sp = []
    try:
        for met in model_sp.metabolites:
            met_sp.append(met.y)
    except Exception:
        model_sp.change_objective('DM_atp_c')
        model_sp.optimize(solver='gurobi').f
        for met in model_sp.metabolites:
            met_sp.append(met.y)
        print unique_rxns[i]
    # setup a zero cutoff
    for ind,item in enumerate(met_sp):
        if -10**-6<item<10**-6:
            met_sp[ind] = 0
    # get list of biomass metabolites
    biomass_mets = []
    for met in model_sp.reactions.get_by_id('biomass_NCI60').metabolites:
        biomass_mets.append(str(met))
        
    # get a list of the non-zero shadow prices and cross-reference biomass metabolites to find limiting nutrient(s) if any
    possible_limiting_nutrients = []
    limiting_nutrients = []
    for ind,item in enumerate(met_sp):
        if item !=0:
            possible_limiting_nutrients.append(str(model_sp.metabolites[ind]))
            possible = str(model_sp.metabolites[ind])
            if possible in biomass_mets:
                limiting_nutrients.append(possible)
    # get a list of new objective reactions which should be the corresponding exchange reaction(s) if any
    if limiting_nutrients:
        new_objective_reactions = []
        for item in limiting_nutrients:
            item = item.replace(item[-1],'e')
            item = 'EX_' + item
            new_objective_reactions.append(item)
        run[unique_ent[i]] = new_objective_reactions   
        #### DETERMINING NEW UPTAKE ####
        for n in range(len(new_objective_reactions)):
            model_new_up = model_KO.copy()
            model_new_up.objective = model.reactions.get_by_id(new_objective_reactions[n])
            model_new_up.reactions.get_by_id(new_objective_reactions[n]).lower_bound = -1000
            new_up = model_new_up.optimize(solver='gurobi').f
            model.reactions.get_by_id(new_objective_reactions[n]).lower_bound = new_up
            
            
    newmet_array = [0] * len(model.metabolites)
    met_count = 0
    for i in range(len(model.metabolites)):
        newmet_array[met_count] = string.replace(model.metabolites[i].id, " ", "_")
        met_count = met_count + 1
        
    #####ALGORITHM################
    # creates an empty dict to store DM_rxns as keys and the p_solution.f values
    normal_maxmet_dict = {}

    for i in range(len(model.metabolites)):
        #for i in range(80):

        #  switch toggle
        switch = True

        # this part creates a demand reaction for each metabolite
        temp_DM = 'DM_' + str(newmet_array[i])

        # this nested for loop tries to catch if a demand reaction already exists in the model
        for ii in model.reactions:
            if temp_DM == str(ii):
                # this part sets the newly created demand as the objective
                my_objective = model.reactions.get_by_id(str(temp_DM))
                model.change_objective(my_objective)

                # this part runs FBA on the new demand
                p_solution = model.optimize(solver='gurobi')

                # this stores the demand flux into a dictionary where its demand rxn is the key
                normal_maxmet_dict[str(temp_DM)] = p_solution.f

                # a toggle to see if the reaction is to be deleted or not
                switch = False

        if switch is True:
            temp_DM = Reaction(str(temp_DM))
            newmet_array[i] = model.metabolites.get_by_id(model.metabolites[i].id)
            temp_DM.add_metabolites({newmet_array[i]: -1})
            model.add_reaction(temp_DM)

            # this part sets the newly created demand as the objective
            my_objective = model.reactions.get_by_id(str(temp_DM))
            model.change_objective(my_objective)

            # this part runs FBA on the new demand
            p_solution = model.optimize(solver='gurobi')

            # this stores the demand flux into a dictionary where its demand rxn is the key
            normal_maxmet_dict[str(temp_DM)] = p_solution.f

            # this deletes the demand at the end of each iteration
            model.reactions.get_by_id(str(temp_DM)).delete()


In [32]:
for thing in unique_rxns[0]:
    print thing

SUCD1m


In [33]:
p_solution

<Solution 'infeasible' at 0xd9c9320>

In [34]:
p_solution_ref

<Solution 0.02 at 0x40361d0>

In [35]:
my_objective

<Reaction DM_h_i at 0xef1a208>

In [17]:
# This combines the normal_flux solutions and KO_flux solutions into one dictionary
# The same key, the reaction, and stores 2 flux values as a list to the dic key where [0] is normal and [1] is KO

combined_dic = {key: {'init_flux': normal_maxmet_dict[key], 'KO_flux': KO_maxmet_dict[key]}
                for key in KO_maxmet_dict if key in normal_maxmet_dict}

In [25]:
normal_maxmet_dict

{'DM_vitd3_e': None,
 'DM_3mox4hoxm_c': None,
 'DM_malcoa_m': None,
 'DM_vitd3_c': None,
 'DM_vitd3_m': None,
 'DM_fald_x': None,
 'DM_malcoa_c': None,
 'DM_R1coa_hs_c': None,
 'DM_m3mpdol_U_c': None,
 'DM_adprib_m': None,
 'DM_adprib_c': None,
 'DM_f1p_c': None,
 'DM_R1coa_hs_r': None,
 'DM_for_n': None,
 'DM_gdpddman_c': None,
 'DM_adprib_e': None,
 'DM_3mob_c': None,
 'DM_3mob_m': None,
 'DM_m_em_3gacpail_hs_r': None,
 'DM_od2coa_m': None,
 'DM_ksii_core2_deg3_l': None,
 'DM_od2coa_c': None,
 'DM_quln_c': None,
 'DM_5mthf_e': None,
 'DM_ksii_core2_deg1_l': None,
 'DM_5mthf_c': None,
 'DM_b2coa_m': None,
 'DM_b2coa_x': None,
 'DM_ksii_core2_deg7_l': None,
 'DM_m5mpdol__L_r': None,
 'DM_q10_c': None,
 'DM_pail4p_hs_r': None,
 'DM_acgam6p_c': None,
 'DM_dedol__L_c': None,
 'DM_3dpdhb_me_m': None,
 'DM_prostgi2_r': None,
 'DM_acglcgalacglcgal14acglcgalgluside_hs_g': None,
 'DM_3hbcoa_m': None,
 'DM_5mdr1p_c': None,
 'DM_3mop_m': None,
 'DM_xoltri27_e': None,
 'DM_3mop_c': None,
 'DM_3hb

In [18]:
G1 = {}
G2 = {}
G3 = {}

for rxn, flux in combined_dic.iteritems():
    # if both init and final are 0, stored into group 1
    if flux['init_flux'] == 0 and flux['KO_flux'] == 0:
        G1[rxn] = {'init_flux': flux['init_flux'], 'KO_flux': flux['KO_flux']}
        
    # if both init and final are non-zero, stored into group 1
    if flux['init_flux'] != 0 and flux['KO_flux'] != 0:
        G1[rxn] = {'init_flux': flux['init_flux'], 'KO_flux': flux['KO_flux']}
        
    # if init is 0 and final is non-zero, stored into group 2
    if flux['init_flux'] == 0 and flux['KO_flux'] != 0:
        G2[rxn] = {'init_flux': flux['init_flux'], 'KO_flux': flux['KO_flux']}
        
    # if init is non-zero and final is 0, stored into group 3
    if flux['init_flux'] != 0 and flux['KO_flux'] == 0:
        G3[rxn] = {'init_flux': flux['init_flux'], 'KO_flux': flux['KO_flux']}

In [19]:
# Execute divisions using this function
# This catches divisions of denominator 0 and just outputs 0
def safe_div(x,y):
    if y == 0:
        return 0
    return x / y

In [21]:
# This is the first step of normalization
# This is the flux of KO - initial flux
G1_output_norm_dic = {a: {'init_flux': b['init_flux'], 'KO_flux': b['KO_flux'], 'normalized_flux':
                       safe_div((b['KO_flux']-b['init_flux']),b['init_flux']), 
                          'absolute_normalized_flux': abs(safe_div((b['KO_flux']-b['init_flux']),b['init_flux']))} for a,b in G1.iteritems()}

TypeError: unsupported operand type(s) for -: 'float' and 'NoneType'

In [22]:
G1

{'DM_vitd3_e': {'KO_flux': 0.03465435036923736, 'init_flux': None},
 'DM_3mox4hoxm_c': {'KO_flux': 0.0051203531999999274, 'init_flux': None},
 'DM_malcoa_m': {'KO_flux': 0.2384828829175525, 'init_flux': None},
 'DM_4fumacac_c': {'KO_flux': 0.005120353199999999, 'init_flux': None},
 'DM_vitd3_c': {'KO_flux': 0.03465435038529603, 'init_flux': None},
 'DM_dttp_m': {'KO_flux': 0.1437884719343856, 'init_flux': None},
 'DM_vitd3_m': {'KO_flux': 0.034654349787160754, 'init_flux': None},
 'DM_for_c': {'KO_flux': 1.3172472902950016, 'init_flux': None},
 'DM_malcoa_c': {'KO_flux': 0.23502660891266403, 'init_flux': None},
 'DM_R1coa_hs_c': {'KO_flux': 0.06497690571559632, 'init_flux': None},
 'DM_dttp_c': {'KO_flux': 0.14378847077023238, 'init_flux': None},
 'DM_mepi_e': {'KO_flux': 0.005120353199999999, 'init_flux': None},
 'DM_pail4p_hs_n': {'KO_flux': 0.027927682170407857, 'init_flux': None},
 'DM_mepi_c': {'KO_flux': 0.005120353199999999, 'init_flux': None},
 'DM_1pyr5c_m': {'KO_flux': 0.3373

In [23]:
G2

{}

In [16]:
possible_limiting_nutrients

[]

In [86]:
run

{'10026': [],
 '1040': [],
 '10476': [],
 '10560': [],
 '10632': [],
 '10797': [],
 '11343': [],
 '1152': [],
 '128': [],
 '1345': [],
 '1346': [],
 '1349': [],
 '1545': [],
 '1579': [],
 '1645': [],
 '1650': [],
 '1718': ['EX_chsteeol_e'],
 '1738': [],
 '178': [],
 '1809': [],
 '1836': [],
 '1890': [],
 '19': [],
 '2023': [],
 '2026': [],
 '2108': [],
 '216': [],
 '217': [],
 '2184': [],
 '220': [],
 '2222': ['EX_chsteeol_e'],
 '226': [],
 '23236': [],
 '23443': [],
 '2348': [],
 '23657': [],
 '259230': ['EX_sphmyln_hs_e'],
 '2597': [],
 '26': [],
 '262': [],
 '2643': [],
 '2673': [],
 '2745': [],
 '2752': [],
 '2799': [],
 '2821': [],
 '2877': [],
 '2878': [],
 '2879': [],
 '3030': [],
 '3074': [],
 '3098': [],
 '3099': [],
 '3176': [],
 '3417': [],
 '3612': ['EX_pail_hs_e'],
 '3613': ['EX_pail_hs_e'],
 '3945': [],
 '4124': [],
 '4128': [],
 '4129': [],
 '4200': [],
 '4258': [],
 '4259': [],
 '440': [],
 '445': ['EX_arg__L_e'],
 '4513': [],
 '4536': [],
 '4698': [],
 '4720': [],
 '48

In [34]:
run ={}

# essentially for each gene to be knocked out
i=4
# create a copy of reference model so don't have to worry about things accidently adding up
model = model_ref.copy()
#create a demand of the products of the corresponding reactions mapped by the gene
# keep track of the reactions we added to this model so we can delete them for the next round
# alternatively could do something like model_temp = model.copy() or model = model_ref.copy()
reactions_added = []
# for each reaction mapped by the gene
for reaction in unique_rxns[i]:
    print reaction, model.reactions.get_by_id(reaction).reaction
    # iterate through products of the reaction
    if p_solution.x_dict[reaction] > 0:
        for x in model.reactions.get_by_id(str(reaction)).products:
            # create string for demand for further use
            temp_DM = 'DM_' + str(x)
            # create string for metabolite for futher use
            temp_met = model.metabolites.get_by_id(str(x))
            #print temp_met

            # handy toggle
            switch = True

            # check if demand reaction already in model
            for ii in model.reactions:
                # if already in model add the new lower bound to previous lower bound
                if temp_DM == str(ii):
                    model.reactions.get_by_id(str(temp_DM)).lower_bound = model.reactions.get_by_id(str(temp_DM)).lower_bound + p_solution.x_dict[str(reaction)]
                    switch = False
            # if demand not already in the model create the demand and set lower bound to pFBA solution of the reaction
            if switch == True:
                temp_DM = Reaction(str(temp_DM))
                temp_DM.add_metabolites({temp_met: -1})
                model.add_reaction(temp_DM)
                model.reactions.get_by_id(str(temp_DM)).lower_bound = p_solution.x_dict[str(reaction)]
                reactions_added.append(str(temp_DM))
    elif p_solution.x_dict[reaction] < 0:
        for x in model.reactions.get_by_id(str(reaction)).reactants:
            # create string for demand for further use
            temp_DM = 'DM_' + str(x)
            # create string for metabolite for futher use
            temp_met = model.metabolites.get_by_id(str(x))
            #print temp_met

            # handy toggle
            switch = True

            # check if demand reaction already in model
            for ii in model.reactions:
                # if already in model add the new lower bound to previous lower bound
                if temp_DM == str(ii):
                    model.reactions.get_by_id(str(temp_DM)).lower_bound = model.reactions.get_by_id(str(temp_DM)).lower_bound - p_solution.x_dict[str(reaction)]
                    switch = False
            # if demand not already in the model create the demand and set lower bound to pFBA solution of the reaction
            if switch == True:
                temp_DM = Reaction(str(temp_DM))
                temp_DM.add_metabolites({temp_met: -1})
                model.add_reaction(temp_DM)
                model.reactions.get_by_id(str(temp_DM)).lower_bound = -p_solution.x_dict[str(reaction)]
                reactions_added.append(str(temp_DM))
try:
    p_solution = cobra.flux_analysis.parsimonious.optimize_minimal_flux(model)
except ValueError:
    pass
    #out.write('%s\n'%(reactions_added))



RNDR1 adp_c + trdrd_c --> dadp_c + h2o_c + trdox_c
RNDR2 gdp_c + trdrd_c --> dgdp_c + h2o_c + trdox_c
RNDR3 cdp_c + trdrd_c --> dcdp_c + h2o_c + trdox_c
RNDR4 trdrd_c + udp_c --> dudp_c + h2o_c + trdox_c


In [79]:
out.close()

In [121]:
p_solution.x_dict['RNDR4']

0.0

In [ ]:
# for each reaction after performing the healthy algorithm, KO the reaction and KO the demands

In [119]:
reactions_added

['DM_h2o_c', 'DM_trdox_c', 'DM_dadp_c', 'DM_dgdp_c', 'DM_dcdp_c']

In [118]:
p_solution.f

0.0177

In [81]:
model.objective

{<Reaction biomass_NCI60 at 0x121188d0>: 1}

In [42]:
model.reactions.get_by_id('DM_co2_e').lower_bound

1.50298885854687

In [10]:
p_solution.x_dict

{'CSND': 0.0,
 'EX_gdchola_e': 0.0,
 'NADPN': 0.0,
 'NDP7g': 0.0,
 'EX_dgchol_e': 0.0,
 'DM_dttp_n': 0.0,
 'UMPK5n': 0.0,
 'NTD10': 0.0,
 'NTD11': 0.0,
 'NTD12': 0.0,
 'UPPN': 0.0,
 'EX_adprbp_e': 0.0,
 'MCCCrm': 0.0012881516000000006,
 'EX_cca_d3_e': 0.0,
 'G5SDym': 0.004240901000000002,
 'GTHOm': 0.0,
 'MGACONm': 0.0,
 'KYN': 0.0,
 'EX_fucfuc132galacglcgal14acglcgalgluside_hs_e': 0.0,
 'FALDH': 0.0,
 'SPR': 0.0,
 'ITCOAL1m': 0.0,
 'FPGS2m': 0.0,
 'OPAH': 0.0,
 'EX_lac__D_e': 0.0,
 'CSNt': 0.0,
 'ARTPLM1m': 0.0,
 'AG13T9g': 0.0,
 'SPRMS': 0.0,
 'PCHOL_HStg': 0.0,
 'P4502D6': 0.0,
 'CAT2p': 0.0,
 'XOLDIOLONEt': 0.0,
 'DNDPt50m': 0.0,
 'GTHOr': 0.0,
 'RETFAt1': 0.0,
 'LNLNCGt': 0.0,
 'GCPNn': 0.0,
 'GALASE6ly': 0.0,
 'DOLPMT4_Uer': 0.0,
 'ETOHt': 0.0,
 'ACALDt': 0.0,
 'HPYRRy': 0.0,
 'TETPENT6t': 0.0,
 'PTHPS': 0.0,
 'HAtly': 0.0,
 'CO2t': -1.50298885854687,
 'G5SADs': 0.0,
 'TRIODTHYSULT': 0.0,
 'NACASPAH': 0.0,
 'METt4': 0.0,
 'APRTO2': 0.0,
 'PACCOAL': 0.0,
 'INSTt4_2': 0.0,
 'GASNAS

In [56]:
sols = {}

In [57]:
sols['solutions'] = p_solution.x_dict

In [83]:
already_dem = []
for reaction in model.reactions:
    if 'DM_' in str(reaction):
        already_dem.append(str(reaction))

In [100]:
for i in already_dem:
    print i,model.reactions.get_by_id(i).lower_bound

DM_13_cis_oretn_n 0.0
DM_13_cis_retn_n 0.0
DM_Asn_X_Ser_Thr_l 0.0
DM_Ser_Thr_l 0.0
DM_Ser_Gly_Ala_X_Gly_l 0.0
DM_avite1_c 0.0
DM_avite2_c 0.0
DM_bvite_c 0.0
DM_datp_m 0.0
DM_datp_n 0.0
DM_dctp_m 0.0
DM_dctp_n 0.0
DM_dgtp_m 0.0
DM_dgtp_n 0.0
DM_dsT_antigen_g 0.0
DM_dttp_m 0.0
DM_dttp_n 0.0
DM_ethamp_r 0.0
DM_gpi_sig_r 0.0
DM_hretn_n 0.0
DM_kdn_c 0.0
DM_melanin_c 0.0
DM_oretn_n 0.0
DM_sTn_antigen_g 0.0
DM_sprm_c 0.0
DM_yvite_c 0.0
DM_etha_c 0
DM_atp_c 7.19451
DM_lac__L_c 0.513705366409
DM_nad_c 0.513705366409
DM_etha_c_reverse 0


In [95]:
model.reactions.get_by_id('DM_etha_c_reverse').lower_bound

0

In [124]:
unique_unique_rxns = []
for i in unique_rxns:
    if i not in unique_unique_rxns:
        unique_unique_rxns.append(i)

In [140]:
out_file = open('deficient_reactions_and_fluxes.txt','w')
for i in range(len(unique_unique_rxns)):
    cur_rxns = unique_unique_rxns[i]
    fluxes = []
    for j in range(len(cur_rxns)):
        fluxes.append(p_solution.x_dict[cur_rxns[j]])
    out_file.write('%s\t%s\n'%(cur_rxns,fluxes))
out_file.close()

In [43]:
model = model_ref.copy()

In [44]:
p_solution.x_dict['biomass_NCI60']

NameError: name 'p_solution' is not defined

In [45]:
amp_c = model.metabolites.get_by_id('amp_c')
DM_amp_c = Reaction('DM_amp_c')
DM_amp_c.name = DM_amp_c
DM_amp_c.add_metabolites({amp_c: -1})
model.add_reaction(DM_amp_c)

argsuc_c = model.metabolites.get_by_id('argsuc_c')
DM_argsuc_c = Reaction('DM_argsuc_c')
DM_argsuc_c.name = DM_argsuc_c
DM_argsuc_c.add_metabolites({argsuc_c: -1})
model.add_reaction(DM_argsuc_c)

h_c = model.metabolites.get_by_id('h_c')
DM_h_c = Reaction('DM_h_c')
DM_h_c.name = DM_h_c
DM_h_c.add_metabolites({h_c: -1})
model.add_reaction(DM_h_c)

ppi_c = model.metabolites.get_by_id('ppi_c')
DM_ppi_c = Reaction('DM_ppi_c')
DM_ppi_c.name = DM_ppi_c
DM_ppi_c.add_metabolites({ppi_c: -1})
model.add_reaction(DM_ppi_c)

In [46]:
model.reactions.get_by_id('ARGSS').upper_bound = 0
model.reactions.get_by_id('ARGSS').lower_bound = 0

In [47]:
model.reactions.get_by_id('DM_amp_c').upper_bound = 0
model.reactions.get_by_id('DM_amp_c').lower_bound = 0

model.reactions.get_by_id('DM_argsuc_c').upper_bound = 0
model.reactions.get_by_id('DM_argsuc_c').lower_bound = 0

model.reactions.get_by_id('DM_h_c').upper_bound = 0
model.reactions.get_by_id('DM_h_c').lower_bound = 0

model.reactions.get_by_id('DM_ppi_c').upper_bound = 0
model.reactions.get_by_id('DM_ppi_c').lower_bound = 0

In [48]:
model.reactions.get_by_id('biomass_NCI60').lower_bound = 0

In [13]:
model.objective

{<Reaction biomass_NCI60 at 0xe1e1128>: 1}

In [14]:
#p_solution = cobra.flux_analysis.parsimonious.optimize_minimal_flux(model)

In [15]:
model.optimize()

<Solution 0.01 at 0xe1e1da0>

In [16]:
met_sp = []
for met in model.metabolites:
    met_sp.append(met.y)

In [17]:
for ind,item in enumerate(met_sp):
    if -10**-6<item<10**-6:
        met_sp[ind] = 0

In [19]:
biomass_mets = []
for met in model.reactions.get_by_id('biomass_NCI60').metabolites:
    biomass_mets.append(str(met))

In [20]:
possible_limiting_nutrients = []
limiting_nutrients = []
for ind,item in enumerate(met_sp):
    if item !=0:
        possible_limiting_nutrients.append(str(model.metabolites[ind]))
        possible = str(model.metabolites[ind])
        if possible in biomass_mets:
            limiting_nutrients.append(possible)

In [21]:
new_objective_reactions = []
for item in limiting_nutrients:
    item = item.replace(item[-1],'e')
    item = 'EX_' + item
    new_objective_reactions.append(item)

In [32]:
new_objective_reactions[0] == 'EX_arg__L_e'

True

In [23]:
model.change_objective(new_objective_reactions[0])

In [54]:
model.reactions.get_by_id(new_objective_reactions[0]).lower_bound = -1000

In [53]:
model.reactions.get_by_id('biomass_NCI60').lower_bound

0

In [25]:
model.objective

{<Reaction EX_arg__L_e at 0x1053fcc0>: 1}

In [29]:
sol = model.optimize()

In [30]:
sol.f

0.0

In [55]:
model.objective = model.reactions.get_by_id('EX_arg__L_e')

In [56]:
sol = model.optimize()

In [57]:
sol.f

0.0

In [156]:
for met in model.metabolites:
    print met.y

-0.0
0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
0.0
0.0
0.0
-0.0
0.0
0.0
-0.0
0.0
0.0
0.0
0.0
-0.0
-0.0
-0.0
0.0
0.0
-0.0
-0.0
0.0
-0.0
-0.0
0.0
-0.0
-0.0
-0.0
-0.0
0.0
0.0
0.0
0.0
-0.0
0.0
0.0
0.0
-0.0
0.0
0.0
-0.0
0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
0.0
-0.0
0.0
0.0
0.0
0.0
-0.0
-0.0
-0.0
0.0
0.0
0.0
-0.0
0.0
0.0
0.0
0.0
0.0
0.0
-0.0
-0.0
-0.0
-0.0
0.0
-0.0
-0.0
-0.0
0.0
-0.0
0.0
0.0
0.0
-0.0
-0.0
0.0
0.0
0.0
-0.0
0.0
0.0
-0.0
0.0
-0.0
0.0
0.0
-0.0
-0.0
0.0
-0.0
-0.0
0.0
-0.0
-0.0
0.0
0.0
0.0
-0.0
-0.0
-0.0
0.0
0.0
-0.0
-0.0
0.0
0.0
-0.0
-0.0
-0.0
0.0
-0.0
-0.0
0.0
-0.0
-0.0
0.0
0.0
0.0
0.0
-0.0
-0.0
-0.0
-0.0
0.0
-0.0
0.0
0.0
0.0
0.0
0.0
-0.0
-0.0
-0.0
-0.0
0.0
0.0
-0.0
0.0
-0.0
-0.0
-0.0
-0.0
-0.0
0.0
-0.0
0.0
0.0
-0.0
-0.0
-0.0
-0.0
0.0
-0.0
-0.0
0.0
0.0
-0.0
0.0
0.0
0.0
-0.0
0.0
-0.0
0.0
-0.0
0.0
-0.0
-0.0
-0.0
-0.0
-0.0
0.0
0.0
-0.0
-0.0
-0.0
-0.0
0.0
-0.0
0.0
-0.0
-0.0
-0.0
-0.0
-0.0
0.0
-0.0
0.0
-0.0
0.0
0.0
0.0
-0.0
0.0
0.0
0.0
-0.0
0.0
-0.0
-0.0
-0.0
-0.0
-0.0
0.0
-0.0
0

ValueError: I/O operation on closed file

In [ ]:
model_KO = model.copy()
for reaction in unique_rxns[i]:
    model_KO.reactions.get_by_id(reaction).lower_bound = 0
    model_KO.reactions.get_by_id(reaction).upper_bound = 0
    
for reaction in reactions_added:
    model_KO.reactions.get_by_id(reaction).lower_bound = 0
    model_KO.reactions.get_by_id(reaction).upper_bound = 0
    


In [203]:
out_file = open('ARGSS_KO_shadow_prices.csv','w')
for met in model.metabolites:
    if met.y < 10**-6 or met.y>-10**-6:
        out_file.write('%s,%s\n'%(met,0))
    else:
        out_file.write('%s,%s\n'%(met,met.y))
out_file.close()